# Description

This notebook is used for training the DrowsinessModel (defined in model.py) to detect if a driver is tending to fall asleep.

# Imports

In [1]:
import torch, os
from model import DrowsinessModel
from dataset import DrowsinessDataset
from torch.nn import CrossEntropyLoss
from torch.utils.data import random_split, DataLoader
from torchvision import transforms

# Initializations and Hyperparameters

### Initializations

In [2]:
ANNOTATIONS_PATH = 'dataset_annotations.json'
TRAIN_FRACTION = 0.8

### Model Parameters

In [3]:
IN_CHANNELS = 3
OUT_CHANNELS = 4
FEATURES = [16, 32, 64, 128]

### Training Parameters

In [4]:
EPOCHS = 50
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-6
BATCH_SIZE = 32

# Prerequisites

### Loading dataset

In [5]:
transform = transforms.Compose([
    transforms.Resize((145, 145)),
    transforms.ToTensor(), 
])

In [6]:
full_dataset = DrowsinessDataset(ANNOTATIONS_PATH, transform=transform)
len(full_dataset)

2896

In [7]:
train_size = int(TRAIN_FRACTION*len(full_dataset))
test_size = len(full_dataset)-train_size
train_size, test_size

(2316, 580)

In [8]:
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [9]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [10]:
for feature, target in train_loader:
    print(feature.shape, target.shape)
    break

torch.Size([32, 3, 145, 145]) torch.Size([32, 4])


In [11]:
feature.dtype

torch.float32

### Model

In [12]:
model = DrowsinessModel(in_channel=IN_CHANNELS, out_channel=OUT_CHANNELS, features=FEATURES)
model

DrowsinessModel(
  (conv_layers): ModuleList(
    (0): ConvPiece(
      (conv): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): ConvPiece(
      (conv): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (2): ConvPiece(
      (conv): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (3): ConvPiece(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-

# Training

### Prerequisities

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [14]:
def train_one_epoch(model, dataloader, loss_fn, optimizer, device):

    running_loss = 0
    last_loss = 0

    for i, data in enumerate(dataloader):

        feature, target = data
        feature = feature.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        predictions = model(feature)

        loss = loss_fn(predictions, target)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (i+1)%5==0:
            last_loss = running_loss/5
            print(f'Batch {i+1},  loss: {last_loss}')
            running_loss=0
    return last_loss




In [15]:
def train(model, train_loader, val_loader, loss_fn, optimizer, device, epochs):

    model = model.to(device)

    for epoch in range(epochs):
        
        print(f"EPOCH {epoch+1}")

        model.train(True)

        avg_loss = train_one_epoch(model, train_loader, loss_fn, optimizer, device)

        running_val_loss = 0.0

        model.eval()

        with torch.no_grad():

            for i, vdata in enumerate(val_loader):
              vfeature, vtarget = vdata
              vfeature=vfeature.to(device)
              vtarget=vtarget.to(device)
              vpredictions = model(vfeature)

              vloss = loss_fn(vpredictions, vtarget)
              running_val_loss+=vloss
            avg_vloss = running_val_loss/(i+1)
        
        print(f'LOSS train {avg_loss}. Validation loss: {avg_vloss} \n\n')

        if (epoch+1) % 5 == 0 or epoch==EPOCHS-1:
           checkpoint = {
                'epoch': epoch,  
                'model_state_dict': model.state_dict(), 
                'optimizer_state_dict': optimizer.state_dict(), 
                'avg_loss':avg_loss,
                'avg_vloss':avg_vloss
               
                   }
           
           if not os.path.exists("./checkpoints"):
              os.mkdir("checkpoints")

           torch.save(checkpoint, f'./checkpoints/checkpoint_{epoch+1}.pth')



        


In [16]:
optimizer = torch.optim.Adam(lr=LEARNING_RATE, params=model.parameters(), weight_decay=WEIGHT_DECAY)
loss_fn = CrossEntropyLoss()

In [ ]:
train(model, train_loader, test_loader, loss_fn, optimizer, device, EPOCHS)